In [ ]:
# Importações para classificação de texto
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from collections import Counter
import warnings
warnings.filterwarnings('ignore')

# Scikit-learn
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV, StratifiedKFold
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB, BernoulliNB, GaussianNB
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier, VotingClassifier, AdaBoostClassifier
from sklearn.metrics import (accuracy_score, precision_score, recall_score, f1_score,
                           classification_report, confusion_matrix, roc_auc_score)
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler

# Dados desbalanceados
from imblearn.over_sampling import SMOTE
from imblearn.under_sampling import RandomUnderSampler

# Visualização
import plotly.express as px
import plotly.graph_objects as go

# NLP
import nltk
from textblob import TextBlob

# Datasets
import sys
sys.path.append('..')
from datasets.textos_exemplo import *
from utils.nlp_utils import *

print("🎯 Bibliotecas para classificação carregadas!")


In [ ]:
# Criar dataset sintético de análise de sentimentos
np.random.seed(42)

# Reviews positivos
reviews_positivos = [
    "Excelente produto, recomendo muito!",
    "Adorei a qualidade, superou minhas expectativas.",
    "Muito bom, chegou rápido e bem embalado.",
    "Produto incrível, vale cada centavo!",
    "Estou muito satisfeito com a compra.",
    "Qualidade excepcional, comprarei novamente.",
    "Fantástico! Melhor que imaginava.",
    "Produto maravilhoso, entrega rápida.",
    "Simplesmente perfeito, recomendo a todos.",
    "Excelente custo-benefício, muito bom produto."
]

# Reviews negativos  
reviews_negativos = [
    "Produto terrível, não recomendo.",
    "Muito ruim, dinheiro jogado fora.",
    "Qualidade péssima, chegou quebrado.",
    "Horrível, não serve para nada.",
    "Decepcionante, esperava muito mais.",
    "Produto de baixa qualidade, muito caro.",
    "Não gostei, vou devolver.",
    "Péssimo atendimento e produto ruim.",
    "Não vale o preço, muito caro para pouca qualidade.",
    "Produto defeituoso, não funcionou."
]

# Criar dataset
X = reviews_positivos + reviews_negativos
y = ['positivo'] * len(reviews_positivos) + ['negativo'] * len(reviews_negativos)

# Adicionar dados dos nossos textos de exemplo
X.extend(textos_reviews)
# Simular labels para os reviews (baseado em palavras-chave simples)
for review in textos_reviews:
    palavras_positivas = ['bom', 'ótimo', 'excelente', 'recomendo', 'gostei', 'perfeito']
    palavras_negativas = ['ruim', 'péssimo', 'horrível', 'terrível', 'não gostei', 'decepcionante']
    
    review_lower = review.lower()
    score_pos = sum(1 for palavra in palavras_positivas if palavra in review_lower)
    score_neg = sum(1 for palavra in palavras_negativas if palavra in review_lower)
    
    if score_pos > score_neg:
        y.append('positivo')
    else:
        y.append('negativo')

print(f"📊 Dataset criado:")
print(f"Total de exemplos: {len(X)}")
print(f"Positivos: {y.count('positivo')}")
print(f"Negativos: {y.count('negativo')}")

# Exemplos do dataset
print(f"\n📝 Exemplos do dataset:")
for i in range(5):
    print(f"{y[i]:10}: {X[i][:50]}...")

# Divisão treino/teste
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.3, random_state=42, stratify=y
)

print(f"\n🔀 Divisão dos dados:")
print(f"Treino: {len(X_train)} exemplos")
print(f"Teste:  {len(X_test)} exemplos")

# Função para avaliar modelo
def avaliar_modelo(name, model, X_train, X_test, y_train, y_test):
    """Avalia um modelo de classificação"""
    
    # Treinar
    start_time = time.time()
    model.fit(X_train, y_train)
    train_time = time.time() - start_time
    
    # Predizer
    start_time = time.time()
    y_pred = model.predict(X_test)
    pred_time = time.time() - start_time
    
    # Métricas
    accuracy = accuracy_score(y_test, y_pred)
    precision = precision_score(y_test, y_pred, pos_label='positivo')
    recall = recall_score(y_test, y_pred, pos_label='positivo')
    f1 = f1_score(y_test, y_pred, pos_label='positivo')
    
    print(f"\n📊 {name}")
    print("-" * (len(name) + 4))
    print(f"Accuracy:  {accuracy:.3f}")
    print(f"Precision: {precision:.3f}")
    print(f"Recall:    {recall:.3f}")
    print(f"F1-Score:  {f1:.3f}")
    print(f"Tempo treino: {train_time:.3f}s")
    print(f"Tempo pred:   {pred_time:.3f}s")
    
    return {
        'name': name,
        'accuracy': accuracy,
        'precision': precision,
        'recall': recall,
        'f1': f1,
        'train_time': train_time,
        'pred_time': pred_time,
        'y_pred': y_pred
    }

# Pipeline com diferentes algoritmos
modelos = [
    ('Naive Bayes', Pipeline([
        ('tfidf', TfidfVectorizer(max_features=1000)),
        ('nb', MultinomialNB())
    ])),
    
    ('SVM Linear', Pipeline([
        ('tfidf', TfidfVectorizer(max_features=1000)),
        ('svm', SVC(kernel='linear', random_state=42))
    ])),
    
    ('Logistic Regression', Pipeline([
        ('tfidf', TfidfVectorizer(max_features=1000)),
        ('lr', LogisticRegression(random_state=42))
    ])),
    
    ('Random Forest', Pipeline([
        ('tfidf', TfidfVectorizer(max_features=1000)),
        ('rf', RandomForestClassifier(n_estimators=100, random_state=42))
    ]))
]

# Avaliar todos os modelos
resultados = []
for nome, modelo in modelos:
    resultado = avaliar_modelo(nome, modelo, X_train, X_test, y_train, y_test)
    resultados.append(resultado)

# Comparar resultados
print(f"\n🏆 COMPARAÇÃO DOS MODELOS:")
print("=" * 70)
df_resultados = pd.DataFrame(resultados)
print(df_resultados[['name', 'accuracy', 'f1', 'train_time']].round(3))
